## Introduction to relations and relational algebra
CS 236 <br>
Fall 2025

Michael A. Goodrich and Eric G. Mercer<br>
Brigham Young University <br>
March 2023, Updated Oct 2024 and Sept 2025 <br>
Input from ChatPT-5 in response to prompts on (a) how to introduce parameterized tests in pytest and (b) how to start thinking about test coverage using input partitioning
***

## 0. Overview

This notebook introduces set/relational operations and shows how to write high-quality unit tests in Python (with `pytest`). We will:
- Review the definition of a relation 
- Demonstrate implementations of relations and relational operators in code
- Describe how to construct and use **parameterized tests**
- Discuss **test coverage** via **input partitioning**
- Clarify the difference between **copying** data vs **referencing** it in Python

---

## 1. What is a Relation

The textbook defines as relation as a set of mathematical tuples:

* $R\subset A\times B$
* $R=\{(a,1),(b,2),(c,3),(d,4)\}$

In relational databases, the same relation is represented as a table instead of a set:

| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 
| $d$ | $4$ | 

The first row of the table contains the **attributes** associated with each element of the tuple. We can collecte the attributes into a __header__. You can think of the attributed specified in the header as assigning names to each column. For example, the first column contains elements from the set $A$. The header contains information about the sets from which the cartesian product is formed, $R\subset A\times B$.

Below the header are rows containing __tuples__. The tuples are the elements of the __set__ the defines the relation, $R=\{(a,1),(b,2),(c,3),(d,4)\}$.

Just like the order of elements in a set doesn't matter

* $\{(a,1),(b,2),(c,3),(d,4)\} = \{(d,4),(b,2),(a,1),(c,3)\}$

The order of the rows in the table doesn't matter (except for the header). The following table represents the same relation as the table above.

| char | int | 
| :-: | :-: | 
| $d$ | $4$ | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 


***

## 2. Encoding a Relation as a Python Class

Let's construct a class that represents a relation.  The variables of this function need to match the parts of the table. Thus, there needs to be a variable representing the header and another variable representing the set of tuples. Similar to how the Project 3 started code does it, we'll define the following
- The relation name as a `str`. You won't see the relation name in the starter code for Project 3, but it's useful here since it allows us to track a few things in the tutorial. 
- The relation header will be defined as a list of strings: `list[str]`
- Each tuple in the relation will be defined as a new type called `RelationTuple`
  - The `RelationTuple` will be built using Python's default `tuple` type.  
  - We'll allow both integers and strings to be the elements of a tuple, so look for the vertical bar that says the tuple types can be either an int or a str yielding `RelationTuple = tuple[str | int]`
  - We won't specify how many elements will be in the `RelationTuple` so we'll use Python's `...` to indicate that we can have several tuples yielding `RelationTuple = tuple[str | int, ...]`
- A helper `__str__` method that returns a string in the correct format.

In [85]:
from tabulate import tabulate  # requires the tabulate module

RelationTuple = tuple[str | int, ...]
        
class Relation:
    def __init__(self, relation_name: str, relation_header: list[str], set_of_tuples: set[RelationTuple]) -> None:
        self.name:str = relation_name # I threw in a string for the relation name to help show how the math works
        self.header: list[str] = relation_header
        self.set_of_tuples: set[RelationTuple] = set_of_tuples

    def __str__(self) -> str:
        value: str = f"relation name = {self.name}\n" + tabulate(iter(self.set_of_tuples), self.header, tablefmt="fancy_grid")
        return value
        
    
R: Relation = Relation(relation_name = "R",relation_header = ('char','int'), set_of_tuples = {('a',1),('b',2),('c',3),('d',4)})
print(R)

Q: Relation = Relation('Q',('C','D'),{})
print(Q)

# The following code will cause the __str__ method to crash
# The problem is that tabulate requires strings and the types in the set are ints
# W: Relation = Relation('W',('A'), {1, 2, 3})
# They must be tuples. Later, the trailing comma will be explained
W: Relation = Relation('W',('A'), {(1,), (2,), (3,)})
print(W)


relation name = R
╒════════╤═══════╕
│ char   │   int │
╞════════╪═══════╡
│ a      │     1 │
├────────┼───────┤
│ b      │     2 │
├────────┼───────┤
│ d      │     4 │
├────────┼───────┤
│ c      │     3 │
╘════════╧═══════╛
relation name = Q
╒═════╤═════╕
│ C   │ D   │
╞═════╪═════╡
╘═════╧═════╛
relation name = W
╒═════╕
│   A │
╞═════╡
│   1 │
├─────┤
│   2 │
├─────┤
│   3 │
╘═════╛


The top row in the tables above consist of the relation header. The remaining rows represent the set of tuples contained in the relation. Each line contains a unique tuple.

The order of the tuples in the set of tuples doesn't matter since sets are not ordered, so the relation R2 defined below is the same as R defined above.

---

#### 2.1 The `__eq__` Dunder Method

A **dunder method** is any method in Python that begins and ends with **d**ouble **under**scores. There are several dunder methods defined each time a class is defined. Sometimes, we need to change the default definition of the dunder methods that are automatically created so that they behave like we want. It will be necessary to do this for the dunder method `__eq__` becaue the default implementation does not correctly check whether two relations are equal. 

Consider whether you think the following statement that checks whether two relations are equal should be true or false.

In [86]:
R2: Relation = Relation(relation_name = "R",relation_header = ('char','int'), set_of_tuples = {('a',1),('b',2),('c',3),('d',4)})
print(R)
print(R2)
print(f"The default __eq__ method says the statement R==R2 is {R==R2}")

relation name = R
╒════════╤═══════╕
│ char   │   int │
╞════════╪═══════╡
│ a      │     1 │
├────────┼───────┤
│ b      │     2 │
├────────┼───────┤
│ d      │     4 │
├────────┼───────┤
│ c      │     3 │
╘════════╧═══════╛
relation name = R
╒════════╤═══════╕
│ char   │   int │
╞════════╪═══════╡
│ a      │     1 │
├────────┼───────┤
│ b      │     2 │
├────────┼───────┤
│ d      │     4 │
├────────┼───────┤
│ c      │     3 │
╘════════╧═══════╛
The default __eq__ method says the statement R==R2 is False


Mathematically, $R=R2$ since (a) the headers are equal and (b) the two relations have the same set of tuples. The reason the default implementation of `__eq__` says that R does not equal R2 is the default implementation **doesn't check what's inside the relations** but rather **checks whether R and R2 are two different names for the same object**. In other words, `__eq__` just checks whether the two relation names are the same object.  When we print out the `id` information about each instance, we see that the objects have different addresses.

In [87]:
print(id(R))
print(id(R2))

4431646832
4431645872


We will need to define an equality operator that checks the contents of the relation (both header and set of tuples) rather than just the id's of the two relations. A good discussion of how to do this can be found here: 

https://stackoverflow.com/questions/1227121/compare-object-instances-for-equality-by-their-attributes

Let's redefine the class to define the `__eq__` function.

In [88]:
from typing import Any

class Relation:
    def __init__(self, relation_name: str, relation_header: list[str], set_of_tuples: set[RelationTuple]) -> None:
        self.name:str = relation_name # I threw in a string for the relation name for fun
        self.header: list[str] = relation_header
        self.set_of_tuples: set[RelationTuple] = set_of_tuples

    def __str__(self) -> str:
        value: str = f"relation name = {self.name}\n" + tabulate(iter(self.set_of_tuples), self.header, tablefmt="fancy_grid")
        return value

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, Relation):
            print("you are trying to compare a relation to something that is not a relation")
            return False
        return (
            self.header == other.header
            and self.set_of_tuples == other.set_of_tuples
        )

    
R: Relation = Relation(relation_name = "R",relation_header = ('char','int'), set_of_tuples = {('a',1),('b',2),('c',3),('d',4)})
R2: Relation = Relation(relation_name = "R2",relation_header = ('char','int'), set_of_tuples = {('b',2),('a',1),('c',3),('d',4)})
print(R==R2)

True


Defining the `__eq__` function within the class allows us to use the == operator to check whether the contents of the two relations are the same. Notice a few things about the `__eq__` method:

- Two arguments are passed to the `__eq__` method: `self` and `other`. 
  - `self` refers to the instance of the relation class
  - `other` refers to whatever the relation can be compared to. The code above uses the `Any` type to say that we can compare anything to the relation.
- The first line of the method tests whether the `other` object passed in is a `Relation` type. If it's not, the function returns false.
- The `__eq__` method returns the boolean expression that tests whether both headers match as well as whether the set of tuples match.

***


#### 2.2 Copy vs Reference (Aliasing)

In Python, variables **hold references** to objects. Assigning one variable to another **does not make a new copy** — both names refer to the **same object** (aliasing). Mutating through either name changes the single shared object.

**Ways to copy (make a distinct object):**
- Shallow copy for built-ins:
  - List: `lst.copy()`, `list(lst)`, `lst[:]`
  - Dict: `d.copy()`, `dict(d)`
  - Set: `s.copy()`, `set(s)`
- `copy.copy(x)` for a shallow copy of general objects
- `copy.deepcopy(x)` for a deep copy when nested objects must be copied too

**Rules of thumb**
- If you plan to **mutate**, decide whether that mutation should be **shared** (reference) or **isolated** (copy).
- For **immutable** types (e.g., `int`, `str`, `tuple` of immutables), aliasing is harmless because the object itself cannot be mutated.

Below we demonstrate aliasing vs copying for a list and show the same idea for sets and dicts.

####  2.3 Sometimes, Copying Creates New Objects

Let's start with how some of our intuitions think copying should work in Python.

In [89]:
string1: str = "Happy"
string2: str = string1 + " Birthday"

print(string1)
print(string2)
print(string1 == string2)
print(id(string1))
print(id(string2))

Happy
Happy Birthday
False
4425998784
4432820784


When we copy a string to another string, the copy creates a new instance of the string. When we modify the copy, only the copy is changed.  We showed three pieces of evidence to support this:
- we printed both strings and saw that they were different
- we used `==` to show that the strings were different
- we printed the `id` of both strings and showed that they were different


#### 2.4 Sometimes, Copying Only Copies a Reference

Let's see what happens when we copy a relation. We'll create a copy and print the `id` of both the original and the copy.

In [90]:
R: Relation = Relation(relation_name = "R",relation_header = ('char','int'), set_of_tuples = {('a',1),('b',2),('c',3),('d',4)})
Q: Relation = R

print(id(R))
print(id(Q))

4432623232
4432623232


Notice that the two objects have the same `id`. The problem is that Python defaults to creating a **reference** to the original object. This means that if we modify the copy we also modify the original. We can see this if I overwrite the header of the copy and then print out the header of the original and the copy. 

In [91]:
print(f'Header of R before modification = {R.header}')
print(f'Header of Q before modification = {Q.header}')

Q.header = ['bart', 'lisa']

print(f'Header of R after modification = {R.header}')
print(f'Header of Q after modification = {Q.header}')


Header of R before modification = ('char', 'int')
Header of Q before modification = ('char', 'int')
Header of R after modification = ['bart', 'lisa']
Header of Q after modification = ['bart', 'lisa']


Notice how both the copy and the original are modified. This is because the copy created a new pointer to the original relation, so when we modified the memory to which the pointer referenced the we also modified the original.

#### 2.5 You Can Define Your Own Form of Copy

We can fix this by defining our own `__copy__` method. A good LLM prompt to explore more is:

     is there a dunder method in python that describes how to perform a copy operation of an object


#### 2.6 Python Tools for Making Deep Copies

Python's `deepcopy` creates a new instance of the object.

In [92]:
from copy import deepcopy

R: Relation = Relation(relation_name = "R",relation_header = ('char','int'), set_of_tuples = {('a',1),('b',2),('c',3),('d',4)})
Q: Relation = deepcopy(R)

print(id(R))
print(id(Q))

print(f'Header of R before modification = {R.header}')
print(f'Header of Q before modification = {Q.header}')

Q.header = ['bart', 'lisa']

print(f'Header of R before modification = {R.header}')
print(f'Header of Q before modification = {Q.header}')

Q.header = ['bart', 'lisa']

print(f'Header of R after modification = {R.header}')
print(f'Header of Q after modification = {Q.header}')

4426134064
4431657008
Header of R before modification = ('char', 'int')
Header of Q before modification = ('char', 'int')
Header of R before modification = ('char', 'int')
Header of Q before modification = ['bart', 'lisa']
Header of R after modification = ('char', 'int')
Header of Q after modification = ['bart', 'lisa']


---


## 3. Notes on the Starter Code

There are a few things in the starter code that have been confusing to students in previous semesters. 

#### 3.1  Creating New Initalized Lists

**Note that some of the information shared here was culled from copilot in response to a prompt on differences in initalizing member variables in the starter code.**

Starter code is provided for project 3. It has a subtle but important difference with the definition of the class above. Specifically, assigning the `relation_header` to the class variable `self.header`, which occurs in `__init__`. In the code above, the assignment is

`self.header: list[str] = relation_header`

but in the starter code the assigment is 

`self.header = list(relation_header)`

What's the difference? Let's experiment with some variables outside of the definition of the Relation class. 

In [93]:
relation_header: list[str] = ['A', 'B', 'C']
header_v1: list[str] = relation_header
header_v2 = list(relation_header)

print(f"The original list has id = {id(relation_header)}")
print(f"The assignment operator in version 1 has id = {id(header_v1)}")
print(f"The assignment operator in version 2 has id = {id(header_v2)}")

The original list has id = 4438924096
The assignment operator in version 1 has id = 4438924096
The assignment operator in version 2 has id = 4438768064


Notice that the id's for the original list and the first version of the header have the same id, but the id's of the original list and the second version of the header have different id's. Stated simply,
 - the assignment `header_v1: list[str] = relation_header` copies a *reference* (e.g., the pointer)
 - the assignment `header_v2 = list(relation_header)` creates a brand new list object

You can tell that the second assignment creates a brand new list object because it uses the formatting that we use whenever we create a new object in Python. The `list(stuff)` creates a new instance of a list object, where `stuff` initializes the list object. We can confirm that a new list object is created by asking Python about the list keyword, as follows:

In [94]:
help(list)

Help on class list in module builtins:

class list(object)
 |  list(iterable=(), /)
 |
 |  Built-in mutable sequence.
 |
 |  If no argument is given, the constructor creates a new empty list.
 |  The argument must be an iterable if specified.
 |
 |  Methods defined here:
 |
 |  __add__(self, value, /)
 |      Return self+value.
 |
 |  __contains__(self, key, /)
 |      Return bool(key in self).
 |
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |
 |  __eq__(self, value, /)
 |      Return self==value.
 |
 |  __ge__(self, value, /)
 |      Return self>=value.
 |
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |
 |  __getitem__(self, index, /)
 |      Return self[index].
 |
 |  __gt__(self, value, /)
 |      Return self>value.
 |
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |
 |  __imul__(self, value, /)
 |      Implement self*=value.
 |
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate sign

---

#### 3.2 Creating relations with one-ples or empty relations ####

A tuple can have only a single element in it, what Goodrich jokingly calls a "one-ple" in class. There is a tricky problem that comes up when we try to create a tuple with only a single element. Consider the following:

In [95]:

R: Relation = Relation('R',('char'),{('a')}) #To create a tuple with only one item, you have add a comma after the item, otherwise Python will not recognize the variable as a tuple.
print(R)


relation name = R
╒═════╕
│ c   │
╞═════╡
│ a   │
╘═════╛


Notice how the only the "c" from the attribute name "char" is printed out. This can be a difficult bug to diagnose.

It gets worse if we try to create a relation with no tuples in the set

In [96]:
Q: Relation = Relation('Q',('char'),{}) #To create a tuple with only one item, you have add a comma after the item, otherwise Python will not recognize the variable as a tuple.
print(Q)

relation name = Q
╒═════╤═════╤═════╤═════╕
│ c   │ h   │ a   │ r   │
╞═════╪═════╪═════╪═════╡
╘═════╧═════╧═════╧═════╛


That's not what we intended at all. We can get around this by adding a comma after the single element of the tuple (after both the 'char' and the 'a')...

In [97]:
P: Relation = Relation('P',('char',),{('a',)}) #To create a tuple with only one item, you have add a comma after the item, otherwise Python will not recognize the variable as a tuple.
print(P)


relation name = P
╒════════╕
│ char   │
╞════════╡
│ a      │
╘════════╛


... and by adding a single comma after the header when we create a relation with the set of tuples empty

In [98]:
Q: Relation = Relation('Q',('char',),{}) #To create a tuple with only one item, you have add a comma after the item, otherwise Python will not recognize the variable as a tuple.
print(Q)


relation name = Q
╒════════╕
│ char   │
╞════════╡
╘════════╛


A good LLM prompt to understand this issue better is

     Why do i have to add a comma after a list with only one element in python? 

--- 

## 4. Adding relational algebra operators to the Relation class

We are now in a position to start filling in the rest of the Relation class.

I want to be able to apply the _relational operators_ to any relation or pair of relations. Using good object-oriented programming style, I'll add the relational operators as functions to the class.

Let's begin by creating our own error handler a la the starter code


In [99]:
class IncompatibleOperandError(Exception):
    def __init__(self, msg: str) -> None:
        super().__init__(msg)

#### 4.1 Union

Consider the relation R defined as before
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 
| $d$ | $4$ | 

and consider a new relation Q defined as
| char | int | 
| :-: | :-: | 
| $f$ | $3$ |

The union $P\cup Q$ is possible since the headers match, and is
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 
| $d$ | $4$ | 
| $f$ | $3$ |  

Let's check in the code.

In [114]:
class Relation:
    def __init__(self, relation_name: str, relation_header: list[str], set_of_tuples: set[RelationTuple]) -> None:
        self.name:str = relation_name # I threw in a string for the relation name for fun
        self.header: list[str] = relation_header
        self.set_of_tuples: set[RelationTuple] = set_of_tuples

    def __str__(self) -> str:
        value: str = f"relation name = {self.name}\n" + tabulate(iter(self.set_of_tuples), self.header, tablefmt="fancy_grid")
        return value

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, Relation):
            print("you are trying to compare a relation to something that is not a relation")
            return False
        return (
            self.header == other.header
            and self.set_of_tuples == other.set_of_tuples
        )
    
    ########################
    # Relational Operators #
    ########################
    def union(self,other) -> 'Relation':
        if not isinstance(other, Relation):
            raise IncompatibleOperandError(f"Tried to union a relation with a {type(other)}") # don't attempt to union with something not a relation
        # First, check the precondition to see if the headers are the same
        if self.header != other.header:
            raise IncompatibleOperandError("Tried to union two relations with different headers")
        
        # Second, create a new header that is the union of the sets of tuples
        name: str = self.name + "\u222A" + other.name
        header: list[str] = self.header
        set_of_tuples = self.set_of_tuples
        set_of_tuples = set_of_tuples.union(other.set_of_tuples)    # This is the union operator defined for set objects
        return Relation(name,header,set_of_tuples) # Create a new relation
        


In [115]:
    
R = Relation(relation_name = 'R',relation_header = ('char','int'), set_of_tuples = {('a',1),('b',2),('c',3),('d',4)})
Q = Relation('Q',('char','int'),{('f',3),}) # Notice the comma after the ('f',3) tuple. This create a set of tuples, with only one element in the set

P: Relation = R.union(Q)
print(P) # I can see that the relation name is a union when I print it out

relation name = R∪Q
╒════════╤═══════╕
│ char   │   int │
╞════════╪═══════╡
│ a      │     1 │
├────────┼───────┤
│ d      │     4 │
├────────┼───────┤
│ c      │     3 │
├────────┼───────┤
│ f      │     3 │
├────────┼───────┤
│ b      │     2 │
╘════════╧═══════╛


#### 4.2 Error Handling Unit Tests

Let's check the error handling

In [116]:
try:
    P.union('mystring')
except IncompatibleOperandError as e:
    print(e) 

Tried to union a relation with a <class 'str'>


Let's try to form the union of two relations that have different headers.

In [117]:
W: Relation = Relation('Q',('bart','lisa'),{('f',3),}) # Notice the comma after the ('f',3) tuple. This create a set of tuples, with only one element in the set
try:
    P.union(W)
except IncompatibleOperandError as e:
    print(e) 

Tried to union two relations with different headers


Notice that when we perform the union we don't add the tuples to R. Instead, we create a new set of tuples and return a brand new relation with those tuples.

#### 4.3 Parameterized Unit Tests

There are times when we want to write several tests for a specific function or class. It can become tedious to write a function for each test that we want to perform. A common way around this is to use parameterized tests. 

Instead of writing many tests that are nearly identical, we can write **one test** and feed it **many cases** using `@pytest.mark.parametrize`. This improves readability and coverage with less code. Notice the spelling of `parametrize`. The verb form of **parameter** is spelled **parameterize**. The function used in `pytest` for doing parmeterized tests uses a different spelling. 

When we "do the math" we identify several cases that could cause problems. 
- Disjoint sets
- Overlapping sets
- Union with empty set
- Idempotence: `A ∪ A == A`
- Commutativity: `A ∪ B == B ∪ A`

Let's set up `pytest` inside of the Jupyter notebook before looking at how we'd write parameterized tests.

In [118]:
import ipytest
import pytest

ipytest.autoconfig()


# --- Helpers ---------------------------------------------------------------

def make_rel(name: str, header: list[str], rows: set[tuple]) -> Relation:
    # If your code defines a RelationTuple alias, plain tuples should still work.
    return Relation(name, header, set(rows))


In [119]:
%%ipytest
@pytest.mark.parametrize(
    "hdr,rows1,name1,rows2,name2,expected_rows",
    [
        # Disjoint
        (["A","B"], {(1,"a"), (2,"b")}, "R1",
                     {(3,"c")},            "R2",
                     {(1,"a"), (2,"b"), (3,"c")}),

        # Overlap (duplicates should be removed by set semantics)
        (["A","B"], {(1,"a"), (2,"b")}, "R1",
                     {(2,"b"), (3,"c")}, "R2",
                     {(1,"a"), (2,"b"), (3,"c")}),

        # Left empty
        (["A","B"], set(),              "Empty",
                     {(1,"a")},         "R2",
                     {(1,"a")}),

        # Right empty
        (["A","B"], {(1,"a")},          "R1",
                     set(),             "Empty",
                     {(1,"a")}),

        # Idempotent: R ∪ R = R
        (["A","B"], {(10,"x"), (11,"y")}, "R",
                     {(10,"x"), (11,"y")}, "R_again",
                     {(10,"x"), (11,"y")}),
    ],
)
def test_union_basic_cases(hdr, rows1, name1, rows2, name2, expected_rows):
    r1 = make_rel(name1, hdr, rows1)
    r2 = make_rel(name2, hdr, rows2)

    out = r1.union(r2)

    # Header must match original header (by precondition)
    assert out.header == hdr
    # Row set must match expected
    assert out.set_of_tuples == expected_rows
    # Equality should work via __eq__
    assert out == make_rel("EXPECTED", hdr, expected_rows)



.....                                                                                        [100%]
5 passed in 0.04s


#### 4.4 What’s Happening in the Parameterized Tests?

The `@pytest.mark.parametrize` **decorator** is a way to tell `pytest`:  
“Run this same test function many times, once for each row in the table of inputs.” You don't need to understand how **decorators** work, but they are a neat programming concept that allows you to modify the way functions work. A good LLM will be able to provide a tutorial about decorators.

Let’s unpack what’s going on in `test_union_basic_cases`:

```python
@pytest.mark.parametrize(
    "hdr,rows1,name1,rows2,name2,expected_rows",
    [
        (["A","B"], {(1,"a"), (2,"b")}, "R1",
                     {(3,"c")},          "R2",
                     {(1,"a"), (2,"b"), (3,"c")}),   # disjoint

        (["A","B"], {(1,"a"), (2,"b")}, "R1",
                     {(2,"b"), (3,"c")}, "R2",
                     {(1,"a"), (2,"b"), (3,"c")}),   # overlap

        ...
    ],
)
def test_union_basic_cases(hdr, rows1, name1, rows2, name2, expected_rows):
    ...
```

The first argument to `parametrize` is a comma-separated string of variable names.  
These become the parameters to the test function (`hdr, rows1, name1, ...`).

- The second argument is a **list of tuples**.  
  Each tuple is one set of inputs that fills in those variables.

- When pytest runs:
  - It substitutes the first tuple into the variables and runs the test once.  
  - Then it substitutes the second tuple and runs again.  
  - And so on for every row.  

This way, one test function actually checks **five different scenarios**:
1. Disjoint relations  
2. Overlapping relations (testing duplicate elimination)  
3. Left operand empty  
4. Right operand empty  
5. Idempotence (`R ∪ R = R`)  


**Why Do This?**

Without parameterization, we would need to write **five almost-identical test functions**. That makes tests longer, harder to read, and more likely to miss a case.  

By using `parametrize`, we:
- Capture the essence of what changes across cases (the data, not the logic).  
- Keep the test logic short and clear.  
- Guarantee consistent checks across many scenarios.  


**Key takeaway:** Parameterized tests let you explore a whole *family* of inputs with one concise piece of code.

#### 4.5 More Parameterized Tests for Union

Some of the **edge cases** in the math don't fit nicely into the previous parameterized test, but we can collect them into a new decorated test.

In [120]:
%%ipytest
@pytest.mark.parametrize(
    "hdr,rows1,rows2",
    [
        (["A","B"], {(1,"a"), (2,"b")}, {(2,"b"), (3,"c")}),
        (["A"],      {(1,)},            {(1,), (2,)}),
        (["X","Y"], {(0,"p")},          {(0,"p")}),
    ],
)
def test_union_commutative(hdr, rows1, rows2):
    r1 = make_rel("R1", hdr, rows1)
    r2 = make_rel("R2", hdr, rows2)

    # Commutativity: r1 ∪ r2 == r2 ∪ r1
    out1 = r1.union(r2)
    out2 = r2.union(r1)
    assert out1 == out2


def test_union_header_mismatch_raises():
    r1 = make_rel("R1", ["A","B"], {(1,"a")})
    r2 = make_rel("R2", ["A","C"], {(1,"a")})
    with pytest.raises(IncompatibleOperandError):
        _ = r1.union(r2)


def test_union_wrong_type_raises():
    r1 = make_rel("R1", ["A"], {(1,)})
    with pytest.raises(IncompatibleOperandError):
        _ = r1.union({"not": "a relation"})  # type: ignore[arg-type]


def test_union_does_not_mutate_operands():
    hdr = ["A","B"]
    rows1 = {(1,"a")}
    rows2 = {(2,"b")}
    r1 = make_rel("R1", hdr, rows1)
    r2 = make_rel("R2", hdr, rows2)

    _ = r1.union(r2)

    # Ensure original operands unchanged
    assert r1.set_of_tuples == rows1
    assert r2.set_of_tuples == rows2
    assert r1.header == hdr and r2.header == hdr

......                                                                                       [100%]
6 passed in 0.01s


***


## The Project Relational Operator

We'll only implement a portion of this operator since we want to implement the rest in Project 3. Specifically, this project operation will only work to project to a single column.
 
Consider the relation $P = R\cup Q$ defined as before
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 
| $d$ | $4$ |
| $f$ | $3$ | 

We want to compute $\pi_{char}(P)$, which does two things. 
* First, it creates a new relation.
* Second, it populates the new relation with the char column.

The result is the relation $\pi_{char}(P)$
| char | 
| :-: |
| $a$ |
| $b$ |
| $c$ |
| $d$ |
| $f$ |

***
Here's the code

In [121]:
class Relation:
    def __init__(self, relation_name: str, relation_header: list[str], set_of_tuples: set[RelationTuple]) -> None:
        self.name:str = relation_name # I threw in a string for the relation name for fun
        self.header: list[str] = relation_header
        self.set_of_tuples: set[RelationTuple] = set_of_tuples

    def __str__(self) -> str:
        value: str = f"relation name = {self.name}\n" + tabulate(iter(self.set_of_tuples), self.header, tablefmt="fancy_grid")
        return value

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, Relation):
            print("you are trying to compare a relation to something that is not a relation")
            return False
        return (
            self.header == other.header
            and self.set_of_tuples == other.set_of_tuples
        )
    
    ########################
    # Relational Operators #
    ########################
    def union(self,other) -> 'Relation':
        if not isinstance(other, Relation):
            raise IncompatibleOperandError(f"Tried to union a relation with a {type(other)}") # don't attempt to union with something not a relation
        # First, check the precondition to see if the headers are the same
        if self.header != other.header:
            raise IncompatibleOperandError("Tried to union two relations with different headers")
        
        # Second, create a new header that is the union of the sets of tuples
        name: str = self.name + "\u222A" + other.name
        header: list[str] = self.header
        set_of_tuples = self.set_of_tuples
        set_of_tuples = set_of_tuples.union(other.set_of_tuples)    # This is the union operator defined for set objects
        return Relation(name,header,set_of_tuples) # Create a new relation

    def project(self,column_header: str) -> 'Relation':
        # Only a portion of this function is implemented. 
        # Specifically, only the portion that projects onto a single column.
        # You have to implement the rest of this function in the Project 3
        
        # First, check the precondition
        # The precondition for the project operator is that the column attribute
        # must exist in the set of attributes
        if column_header not in set(self.header):
            raise IncompatibleOperandError(f"Tried to project onto {column_header}, which isn't an attribute of the relation")
        
        # Second, create a new relation that is the output of the projection operator.
        # THe relation needs a name, a header, and the set of tuples
        new_name = "\u03C0" + "_{" + column_header + "}(" + self.name + ")" # The \u03c0 is a special code for a union symbol
        new_header = (column_header,) # Notice the comma after "column_header", which forces the header to be a tuple
        header_index = self.header.index(column_header) # Get the index of the header that matches the column you want
        new_tuples: set[RelationTuple] = {(rtuple[header_index],1) for rtuple in self.set_of_tuples}
        new_relation = Relation(new_name,new_header,new_tuples) # Create the relation
        return new_relation 


Let's now look at the output of the project operator. 

In [122]:
R: Relation = Relation(relation_name = 'R',relation_header = ('char','int'), set_of_tuples = {('a',1),('b',2),('c',3),('d',4)})
Q: Relation = Relation('Q',('char','int'),{('f','3'),}) # Observe the comma after the ('f',3) tuple. This forces python to make the tuple the lone element of a set

P: Relation = R.union(Q)
M: Relation = P.project('char')
print(M)


relation name = π_{char}(R∪Q)
╒════╤════════╕
│    │   char │
╞════╪════════╡
│ a  │      1 │
├────┼────────┤
│ b  │      1 │
├────┼────────┤
│ c  │      1 │
├────┼────────┤
│ f  │      1 │
├────┼────────┤
│ d  │      1 │
╘════╧════════╛


Let's project onto the 'int' column instead. Why does projecting onto the 'char' column produce five tuples but projecting onto the 'char' column only produce four tuples?

In [123]:
P = R.union(Q)
M = P.project('int')
print(M)

relation name = π_{int}(R∪Q)
╒════╤═══════╕
│    │   int │
╞════╪═══════╡
│  2 │     1 │
├────┼───────┤
│  3 │     1 │
├────┼───────┤
│  3 │     1 │
├────┼───────┤
│  1 │     1 │
├────┼───────┤
│  4 │     1 │
╘════╧═══════╛


The answer is that the set of tuples is a set, and sets don't have repeats. 

Without ignoring repeats, we have the relation $P$ defined as
| char | int | 
| :-: | :-: | 
| $a$ | $1$ |
| $b$ | $2$ | 
| $c$ | $3$ | 
| $d$ | $4$ |
| $f$ | $3$ | 

yielding $\pi_{int}(P)$ 
| int | 
| :-: | 
| $1$ |
| $2$ | 
| $3$ | 
| $4$ |
| $3$ |

but the "3" appears in the set twice, so the correct answer is
$\pi_{int}(P)$ 
| int | 
| :-: | 
| $1$ |
| $2$ | 
| $3$ | 
| $4$ |


#### 5.1 Test Coverage via Input Partitioning: `project(relation, cols)`

We’ll design tests by **partitioning the input space** into meaningful categories, then pick at least one case from each partition. For projection:

**Operation**: `project(relation: set[tuple], cols: tuple[int, ...]) -> set[tuple]`  
- Keep only the columns in `cols` (by index)
- Remove duplicate tuples after projection

**Partitions to cover:**
1. **Empty relation** (baseline)
2. **Single tuple** (simple)
3. **Multiple tuples, no duplicates after projection**
4. **Multiple tuples, duplicates after projection** (duplicate elimination)
5. **Invalid column index** (error)
6. **Projection to all columns** (should equal original)
7. **Projection to zero columns** (legal? If allowed, result is either `{()}` or empty set depending on definition—here we’ll disallow and raise)

We’ll implement a minimal `project` to support these tests and then write **parameterized tests** covering the partitions.

In [124]:
import pytest

# --- helpers -----------------------------------------------------------------

def make_rel(name: str, header: list[str], rows: set[tuple]) -> Relation:
    # If RelationTuple is an alias to tuple[...] in your code,
    # plain tuples in a set will work.
    return Relation(name, header, set(rows))


# --- tests for Relation.project ----------------------------------------------


In [125]:
%%ipytest
def test_project_single_column_basic() -> None:
    """
    Project onto a single existing column.
    Expect:
      - header becomes that single column
      - rows become 1-tuples
      - duplicates removed by set semantics
    """
    r = make_rel(
        "R",
        ["A", "B"],
        {(1, "x"), (2, "y"), (2, "z")},
    )

    out = r.project("A")

    # Expected relation
    expected = make_rel(
        "\u03C0_{A}(R)",
        ["A"],
        {(1,), (2,)},
    )

    assert out == expected


def test_project_removes_duplicates() -> None:
    """
    When different rows share the same projected value,
    the result contains only one copy of that value.
    """
    r = make_rel(
        "Teams",
        ["player", "team"],
        {("Ann", "Blue"), ("Bob", "Blue"), ("Cyd", "Red")},
    )

    out = r.project("team")

    expected = make_rel(
        "\u03C0_{team}(Teams)",
        ["team"],
        {("Blue",), ("Red",)},
    )

    assert out == expected


def test_project_invalid_column_raises() -> None:
    """Projecting on a non-existent column should raise IncompatibleOperandError."""
    r = make_rel(
        "R",
        ["A", "B"],
        {(1, "x")},
    )
    with pytest.raises(IncompatibleOperandError):
        _ = r.project("C")


def test_project_name_formatting() -> None:
    """
    The resulting relation name should be π_{col}(OriginalName).
    This checks only the name; contents are validated elsewhere.
    """
    r = make_rel("R", ["A", "B"], {(1, "x")})
    out = r.project("A")
    assert out.name == "\u03C0_{A}(R)"


def test_project_different_columns() -> None:
    """
    Smoke test projecting different columns from the same base relation.
    """
    r = make_rel(
        "People",
        ["id", "name"],
        {(1, "Ada"), (2, "Bob")},
    )

    out_id = r.project("id")
    out_name = r.project("name")

    expected_id = make_rel("\u03C0_{id}(People)", ["id"], {(1,), (2,)})
    expected_name = make_rel("\u03C0_{name}(People)", ["name"], {("Ada",), ("Bob",)})

    assert out_id == expected_id
    assert out_name == expected_name

FF..F                                                                                        [100%]
============================================= FAILURES =============================================
_________________________________ test_project_single_column_basic _________________________________

    def test_project_single_column_basic() -> None:
        """
        Project onto a single existing column.
        Expect:
          - header becomes that single column
          - rows become 1-tuples
          - duplicates removed by set semantics
        """
        r = make_rel(
            "R",
            ["A", "B"],
            {(1, "x"), (2, "y"), (2, "z")},
        )
    
        out = r.project("A")
    
        # Expected relation
        expected = make_rel(
            "\u03C0_{A}(R)",
            ["A"],
            {(1,), (2,)},
        )
    
>       assert out == expected
E       assert <__main__.Relation object at 0x1084c9880> == <__main__.Relation object at 0x1084c82

---

You can ignore the rest of this notebook. I'm using it to create material for the slides in class.

#### What is a database? ###

Consider the following datalog program

<pre>
Schemes:
   childOf(P, C)       # P = Parent, C = Child 
   marriedTo(S1, S2)   # S1 = Spouse 1, S2 = Spouse 2

Facts:<br/>
   marriedTo('Jack', 'Jill').
   marriedTo('Annie', 'Aaron').
   marriedTo('Molly', 'Matt').
   childOf('Jack', 'Matt').
   childOf('Annie', 'Molly').   
   childOf('Matt', 'Nate').

Rules:
   childOf(Y, X) :- childOf(Z, X), marriedTo(Y, Z).
   marriedTo(X, Y) :- marriedTo(Y, X).

Queries:
   marriedTo('Jack', 'Jill')?
   childOf('Aaron', 'Molly')? 
   childOf(P, 'Nate')?
</pre>


This datalog program should create a database with two relations. Let's create them. I'm going to do this without having the _name_ variable in the `Relation` class since this matches the starter code more closely.

In [126]:
class Relation:
    def __init__(self, relation_header: list[str], set_of_tuples: set[RelationTuple]) -> None:
        self.header = list(relation_header)
        self.set_of_tuples = set(set_of_tuples)

    def __str__(self) -> str:
        value: str = tabulate(iter(self.set_of_tuples), self.header, tablefmt="fancy_grid")
        return value

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, Relation):
            print("you are trying to compare a relation to something that is not a relation")
            return False
        return (
            self.header == other.header
            and self.set_of_tuples == other.set_of_tuples
        )
    
    ########################
    # Relational Operators #
    ########################
    def union(self,other) -> 'Relation':
        if not isinstance(other, Relation):
            raise IncompatibleOperandError(f"Tried to union a relation with a {type(other)}") # don't attempt to union with something not a relation
        # First, check the precondition to see if the headers are the same
        if self.header != other.header:
            raise IncompatibleOperandError("Tried to union two relations with different headers")
        
        # Second, create a new header that is the union of the sets of tuples
        header: list[str] = self.header
        set_of_tuples = self.set_of_tuples
        set_of_tuples = set_of_tuples.union(other.set_of_tuples)    # This is the union operator defined for set objects
        return Relation(header,set_of_tuples) # Create a new relation

    def project(self,column_header: str) -> 'Relation':
        # Only a portion of this function is implemented. 
        # Specifically, only the portion that projects onto a single column.
        # You have to implement the rest of this function in the Project 3
        
        # First, check the precondition
        # The precondition for the project operator is that the column attribute
        # must exist in the set of attributes
        if column_header not in set(self.header):
            raise IncompatibleOperandError(f"Tried to project onto {column_header}, which isn't an attribute of the relation")
        
        # Second, create a new relation that is the output of the projection operator.
        # THe relation needs a name, a header, and the set of tuples
        new_header = (column_header,) # Notice the comma after "column_header", which forces the header to be a tuple
        header_index = self.header.index(column_header) # Get the index of the header that matches the column you want
        # I modified this next line to produce "one-ples"
        new_tuples: set[RelationTuple] = {(rtuple[header_index],) for rtuple in self.set_of_tuples} 
        new_relation = Relation(new_header,new_tuples) # Create the relation
        return new_relation 

In [127]:
childOf: Relation = Relation(relation_header=['P','C'],
                             set_of_tuples={('Jack', 'Matt'),
                                             ('Annie', 'Molly'),
                                             ('Matt','Nate')})
marriedTo: Relation = Relation(relation_header=['S1','S2'],
                             set_of_tuples={('Jack', 'Jill'),
                                             ('Annie', 'Aaron'),
                                             ('Molly','Matt')})

print(childOf)
print(marriedTo)

╒═══════╤═══════╕
│ P     │ C     │
╞═══════╪═══════╡
│ Annie │ Molly │
├───────┼───────┤
│ Matt  │ Nate  │
├───────┼───────┤
│ Jack  │ Matt  │
╘═══════╧═══════╛
╒═══════╤═══════╕
│ S1    │ S2    │
╞═══════╪═══════╡
│ Jack  │ Jill  │
├───────┼───────┤
│ Annie │ Aaron │
├───────┼───────┤
│ Molly │ Matt  │
╘═══════╧═══════╛


How do I store this in a database? Since we are saying that a database is just a set of relations, let's define a relation object as a dictionary of relations. We'll use the relation's name as the key in the dictionary.

In [128]:
database: dict['str',Relation] = {'childOf': childOf, 'marriedTo': marriedTo}
for relation_name in database.keys():
    print(f"relation {relation_name} is \n{database[relation_name]}")

relation childOf is 
╒═══════╤═══════╕
│ P     │ C     │
╞═══════╪═══════╡
│ Annie │ Molly │
├───────┼───────┤
│ Matt  │ Nate  │
├───────┼───────┤
│ Jack  │ Matt  │
╘═══════╧═══════╛
relation marriedTo is 
╒═══════╤═══════╕
│ S1    │ S2    │
╞═══════╪═══════╡
│ Jack  │ Jill  │
├───────┼───────┤
│ Annie │ Aaron │
├───────┼───────┤
│ Molly │ Matt  │
╘═══════╧═══════╛


Let's create the relation that we know should form when I want to do a project operation. Let's project the _childOf_ relation onto the _P_ column, 

${\rm answer} = \pi_{P}({\rm database['childOf']})$

When we do the math by hand, we know that we should end up with a relation with one column and "one-ples" of the parents.  I'm calling this the _math_answer_ because this is what happens when we do the project operation by hand. We'll create a relation for this answer we obtained by hand.

In [129]:
math_answer: Relation = Relation(relation_header=['P'],
                             set_of_tuples={('Matt',), ('Jack',), ('Annie',)})
print(math_answer)

╒═══════╕
│ P     │
╞═══════╡
│ Annie │
├───────┤
│ Matt  │
├───────┤
│ Jack  │
╘═══════╛


Let's now run the project function from the relation class to test whether the _project_ function we've implemented in code actually works.

In [130]:
code_answer = database['childOf'].project('P')
print(code_answer)

╒═══════╕
│ P     │
╞═══════╡
│ Annie │
├───────┤
│ Matt  │
├───────┤
│ Jack  │
╘═══════╛


And we can put this into an assert statement to test that the code output matches the math answer.

In [131]:
assert code_answer.__str__() == math_answer.__str__(), "Project function not implemented correctly"